# *This file allows to generate all plots necessary for figures*

# **Useful packages and functions**

In [1]:
using DifferentialEquations, Plots, Polynomials, LaTeXStrings, ColorSchemes, DelimitedFiles
using Statistics, StatsPlots, Random, ProgressMeter, Printf, LinearAlgebra
include("FYON_2022_DA_kinetics.jl") # Loading of DA kinetics of gating variables
include("FYON_2022_DA_models.jl") # Loading of DA model
include("FYON_2022_DA_utils.jl") # Loading of some utils functions
include("FYON_2022_DA_gs_derivatives.jl") # Loading of X_inf derivatives
include("FYON_2022_DA_DIC.jl") # Loading of the DIC and compensation algorithm
include("FYON_2022_DA_neuromodulation.jl"); # Loading of the neuromodulation cells functions

# **Global variables**

In [2]:
# Definition of simulation time (in ms)
const Tfinal = 10000
const tspan  = (0.0, Tfinal)

# Definition of reversal potential values (in mV), [Mg] and membrane capacitance
const VNa   = 60. # Sodium reversal potential
const VK    = -85. # Potassium reversal potential
const VCa   = 60. # Calcium reversal potential
const VNMDA = 0. # NMDA reversal potential
const Vleak = -50. # Reversal potential of leak channels
const Mg    = 1.4 # Mg concentration
const C     = 1. # Membrane capacitance

# Definition of voltage range for the DICs
const Vmin = -60 
const Vmax = 0
const V    = range(Vmin, stop=Vmax, step=0.01)

# Definition of the number of cells in the random set
const ncells = 200;

# **Computing PCA of low frequency tonic spiking neurons and homogeneous scaling using Monte Carlo (Figure 1)**

In [3]:
# Initializing some variables
Na_max = 60
Kd_max = 20
CaL_max = 0.1
CaN_max = 0.3
ERG_max = 0.25
leak_max = 0.02
N = 4e5
VV_maxmax = 58.5
VV_maxmin = 54
VV_minmin = -80.5
VV_minmax = -77.5
f_max = 1.95
f_min = 1.75

g_all_MC = readdlm("./data/g_all_MC.dat");

In [4]:
# Number of ion channels in the DA model
nb_channels = 6

# Initializing some variables
g_all_MC_wout_NMDA = zeros(ncells, nb_channels)

# Removing NMDA channels from the PCA analysis since it is constant
j = 1
for i in [1, 2, 3, 4, 5, 7]
    g_all_MC_wout_NMDA[:, j] = deepcopy(g_all_MC[:, i])
    j = j + 1
end

# Means on spiking point cloud
mean_MC = zeros(1, nb_channels)
mean!(mean_MC, g_all_MC_wout_NMDA)

# Defining scaling factors and limits for the plots
s1 = 2
s2 = 2

# Normalizing data before applying PCA
g_all_MC_norm = deepcopy(g_all_MC_wout_NMDA)
for i = 1 : nb_channels
    g_all_MC_norm[:, i] = g_all_MC_wout_NMDA[:, i] / std(g_all_MC_wout_NMDA[:, i])
end

# PCA on spiking point cloud
cov_matrix_MC = cov(g_all_MC_norm)
eig_MC = eigen(cov_matrix_MC, sortby = x -> abs(x))

# Computing PCA components to map back to the non-normalized dataset
eig_MC_plot = deepcopy(eig_MC)
for i = 1 : nb_channels
    eig_MC_plot.vectors[i, :] = eig_MC_plot.vectors[i, :] *
                                        std(g_all_MC_wout_NMDA[:, i])
end

In [8]:
total_var = sum(eig_MC.values)
eig_val_decreasing = reverse(eig_MC.values)

p1a  = plot(eig_val_decreasing, label="", grid=false, yguidefontsize=15, markershape=:diamond, 
            linestyle=:dashdot, xguidefontsize=15, legendfontsize=12, margin=5Plots.mm, 
            xticks=[1, 2, 3, 4, 5, 6], color=:black, markersize=6, linewidth=2, yticks=[0, 3])
ylims!((-0.1, 3))
xlabel!("Component")
ylabel!("Eigenvalue")
display(p1a)
savefig(p1a, "./figures/DA_MC_scree.pdf")

In [10]:
CC1 = scatter_matrix(eig_MC_plot, nb_channels, g_all_MC_wout_NMDA, mean_MC, Na_max, 
                     Kd_max, CaL_max, CaN_max, ERG_max, leak_max, s1, s2, :gray70, :none)
display(CC1)
savefig(CC1, "./figures/DA_MC_matrix.pdf")

In [19]:
# Computing scaling factors for homogeneous scaling
gNa_scaling = mean_MC[1] / mean_MC[6]
gKd_scaling = mean_MC[2] / mean_MC[6]
gCaL_scaling = mean_MC[3] / mean_MC[6]
gCaN_scaling = mean_MC[4] / mean_MC[6]
gERG_scaling = mean_MC[5] / mean_MC[6]

# Computing alignment
scaling = [gNa_scaling/Na_max, gKd_scaling/Kd_max, gCaL_scaling/CaL_max, gCaN_scaling/CaN_max, 
           gERG_scaling/ERG_max, 1/leak_max]
scaling = scaling ./ norm(scaling)

PC1 = eig_MC_plot.vectors[:, nb_channels]
PC1 = [PC1[1]/Na_max, PC1[2]/Kd_max, PC1[3]/CaL_max, PC1[4]/CaN_max, PC1[5]/ERG_max, PC1[6]/leak_max]
PC1 = PC1 ./ norm(PC1)

alignment = abs(PC1' * scaling)
text = @sprintf "Alignement = %.2f%%" alignment*100
display(text)

"Alignement = 91.27%"

In [20]:
# Computing angles
theta_CaLCaN_leak = atan((gCaN_scaling/CaN_max)/
                         (gCaL_scaling/CaL_max))
theta_CaLCaN = atan((eig_MC_plot.vectors[:, nb_channels][4]/CaN_max)/
                    (eig_MC_plot.vectors[:, nb_channels][3]/CaL_max))

theta_CaNERG_leak = atan((gERG_scaling/ERG_max)/
                         (gCaN_scaling/CaN_max))
theta_CaNERG = atan((eig_MC_plot.vectors[:, nb_channels][5]/ERG_max)/
                    (eig_MC_plot.vectors[:, nb_channels][4]/CaN_max))

theta_KdERG_leak = atan((gERG_scaling/ERG_max)/
                        (gKd_scaling/Kd_max))
theta_KdERG = atan((eig_MC_plot.vectors[:, nb_channels][5]/ERG_max)/
                   (eig_MC_plot.vectors[:, nb_channels][2]/Kd_max))

theta_CaLleak_leak = atan((1/leak_max)/
                          (gCaL_scaling/CaL_max))
theta_CaLleak = atan((eig_MC_plot.vectors[:, nb_channels][6]/leak_max)/
                     (eig_MC_plot.vectors[:, nb_channels][3]/CaL_max));

In [22]:
p1b = plot([theta_CaLCaN, theta_CaLCaN], [0, 1], proj=:polar, linewidth=3, label="", yticks=[], 
            yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm, 
            color=1, xtickfontsize=10)
plot!([theta_CaLCaN_leak, theta_CaLCaN_leak], [0, 1], proj=:polar, linewidth=3, label="", 
      color=1, linestyle=:dashdotdot)
plot!([theta_CaNERG, theta_CaNERG], [0, 1], proj=:polar, linewidth=3, label="", 
      color=2)
plot!([theta_CaNERG_leak, theta_CaNERG_leak], [0, 1], proj=:polar, linewidth=3, label="", 
      color=2, linestyle=:dashdotdot)
plot!([theta_KdERG, theta_KdERG], [0, 1], proj=:polar, linewidth=3, label="", 
      color=3)
plot!([theta_KdERG_leak, theta_KdERG_leak], [0, 1], proj=:polar, linewidth=3, label="", 
      color=3, linestyle=:dashdotdot)
plot!([theta_CaLleak, theta_CaLleak], [0, 1], proj=:polar, linewidth=3, label="", 
      color=4)
plot!([theta_CaLleak_leak, theta_CaLleak_leak], [0, 1], proj=:polar, linewidth=3, label="", 
      color=4, linestyle=:dashdotdot)
ylims!(0, 1)
display(p1b)
savefig(p1b, "./figures/DA_MC_polar_leak.pdf")

In [24]:
p1c = scatter(g_all_MC[:, 3], g_all_MC[:, 4], label="", color=:gray70, grid=false,
              yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm,
              markerstrokewidth=0.)

plot!([mean_MC[3] - s1*eig_MC_plot.vectors[:, nb_channels][3]*eig_MC_plot.values[nb_channels], 
       mean_MC[3] + s1*eig_MC_plot.vectors[:, nb_channels][3]*eig_MC_plot.values[nb_channels]],
       [mean_MC[4] - s1*eig_MC_plot.vectors[:, nb_channels][4]*eig_MC_plot.values[nb_channels], 
       mean_MC[4] + s1*eig_MC_plot.vectors[:, nb_channels][4]*eig_MC_plot.values[nb_channels]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:solid)

plot!([mean_MC[3] - 0.05*gCaL_scaling, 
       mean_MC[3] + 0.05*gCaL_scaling],
       [mean_MC[4] - 0.05*gCaN_scaling, 
       mean_MC[4] + 0.05*gCaN_scaling],
       arrow=false, color=:firebrick1, linewidth=4, label="", linestyle=:dashdotdot)

xlabel!(L"\bar{g}_{CaL} (mS/cm^2)")
ylabel!(L"\bar{g}_{CaN} (mS/cm^2)")
xlims!((0, 0.1))
ylims!((0, 0.3001))
display(p1c)
savefig(p1c, "./figures/neuromod_gCaLCaN_MC.pdf")

# **Computing gs(Vth) isohyperplanes of low frequency tonic spiking neurons using Monte Carlo (Figure 2)**

In [25]:
# Extracting threshold voltage and weighting factors for a specific neuron
(gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak) = g_all_MC[1, :]
(ith, iosc, gf, gs, gu, gin, Istatic) = DICs(V, 0., gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak)
Vth = V[ith]
tau_f = tau_m(Vth)
tau_s = tau_n(Vth)
tau_u = 100. # Fixed because no tau for the ERG current
(wfs_m, wsu_m) = var_contribution(tau_m(Vth), tau_f, tau_s, tau_u)
(wfs_h, wsu_h) = var_contribution(tau_h(Vth), tau_f, tau_s, tau_u)
(wfs_n, wsu_n) = var_contribution(tau_n(Vth), tau_f, tau_s, tau_u)
(wfs_mCaL, wsu_mCaL) = var_contribution(tau_mCaL(Vth), tau_f, tau_s, tau_u)
(wfs_mCaN, wsu_mCaN) = var_contribution(tau_mCaN(Vth), tau_f, tau_s, tau_u)
wfs_oERG = 0.
wsu_oERG = 0.

# Modifying scaling factors
s2 = 1.5

# Computing derivatives of gs(Vth) with respect to maximal ion channel conductances
dgs_dNa = (wsu_m - wfs_m)*3*m_inf(Vth)^2*h_inf(Vth)*(Vth-VNa)*dm(Vth) + 
          (wsu_h - wfs_h)*m_inf(Vth)^3*(Vth-VNa)*dh(Vth)
dgs_dCaL = (wsu_mCaL - wfs_mCaL)*2*mCaL_inf(Vth)*(Vth-VCa)*dmCaL(Vth)
dgs_dCaN = (wsu_mCaN - wfs_mCaN)*(Vth-VCa)*dmCaN(Vth)
dgs_dKd = (wsu_n - wfs_n)*3*n_inf(Vth)^2*(Vth-VK)*dn(Vth)
dgs_dERG = (wsu_oERG - wfs_oERG)*(Vth-VK)*doERG(Vth);

In [26]:
# Computing angles
theta_CaLCaN_DIC = -atan((dgs_dCaL/CaN_max)/(dgs_dCaN/CaL_max))
theta_CaLCaN = atan((eig_MC_plot.vectors[:, nb_channels-1][4]/CaN_max)/
                    (eig_MC_plot.vectors[:, nb_channels-1][3]/CaL_max))

theta_KdCaL_DIC = -atan((dgs_dKd/CaL_max)/(dgs_dCaL/Kd_max))
theta_KdCaL = atan((eig_MC_plot.vectors[:, nb_channels-1][3]/CaL_max)/
                   (eig_MC_plot.vectors[:, nb_channels-1][2]/Kd_max))

theta_KdCaN_DIC = -atan((dgs_dKd/CaN_max)/(dgs_dCaN/Kd_max))
theta_KdCaN = atan((eig_MC_plot.vectors[:, nb_channels-1][4]/CaN_max)/
                   (eig_MC_plot.vectors[:, nb_channels-1][2]/Kd_max))
theta_KdCaN4 = atan((eig_MC_plot.vectors[:, nb_channels-3][4]/CaN_max)/
                    (eig_MC_plot.vectors[:, nb_channels-3][2]/Kd_max))

theta_NaCaL_DIC = -atan((dgs_dNa/CaL_max)/(dgs_dCaL/Na_max))
theta_NaCaL = atan((eig_MC_plot.vectors[:, nb_channels-1][3]/CaL_max)/
                   (eig_MC_plot.vectors[:, nb_channels-1][1]/Na_max))
theta_NaCaL4 = atan((eig_MC_plot.vectors[:, nb_channels-3][3]/CaL_max)/
                    (eig_MC_plot.vectors[:, nb_channels-3][1]/Na_max));

In [28]:
p2a = plot([theta_CaLCaN, theta_CaLCaN], [0, 1], proj=:polar, linewidth=3, label="", yticks=[], 
            yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm, 
            color=1, xtickfontsize=10, linestyle=:dash)
plot!([theta_CaLCaN_DIC, theta_CaLCaN_DIC], [0, 1], proj=:polar, linewidth=3, label="", 
      color=1, linestyle=:dashdotdot)
plot!([theta_KdCaL, theta_KdCaL], [0, 1], proj=:polar, linewidth=3, label="", 
      color=2, linestyle=:dash)
plot!([theta_KdCaL_DIC, theta_KdCaL_DIC], [0, 1], proj=:polar, linewidth=3, label="", 
      color=2, linestyle=:dashdotdot)
plot!([theta_KdCaN, theta_KdCaN], [0, 1], proj=:polar, linewidth=3, label="", 
      color=3, linestyle=:dash)
plot!([theta_KdCaN4, theta_KdCaN4], [0, 1], proj=:polar, linewidth=2, label="", 
      color=3, linestyle=:dot)
plot!([theta_KdCaN_DIC, theta_KdCaN_DIC], [0, 1], proj=:polar, linewidth=3, label="", 
      color=3, linestyle=:dashdotdot)
plot!([theta_NaCaL, theta_NaCaL], [0, 1], proj=:polar, linewidth=3, label="", 
      color=4, linestyle=:dash)
plot!([theta_NaCaL_DIC, theta_NaCaL_DIC], [0, 1], proj=:polar, linewidth=3, label="", 
      color=4, linestyle=:dashdotdot)
plot!([theta_NaCaL4, theta_NaCaL4], [0, 1], proj=:polar, linewidth=2, label="", 
      color=4, linestyle=:dot)
ylims!(0, 1)
display(p2a)
savefig(p2a, "./figures/DA_MC_polar_DIC.pdf")

In [31]:
p2b = scatter(g_all_MC[:, 3], g_all_MC[:, 4], label="", color=:gray70, grid=false,
              yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm,
              markerstrokewidth=0.)

plot!([mean_MC[3] - s2*eig_MC_plot.vectors[:, nb_channels-1][3]*eig_MC_plot.values[nb_channels-1], 
       mean_MC[3] + s2*eig_MC_plot.vectors[:, nb_channels-1][3]*eig_MC_plot.values[nb_channels-1]],
       [mean_MC[4] - s2*eig_MC_plot.vectors[:, nb_channels-1][4]*eig_MC_plot.values[nb_channels-1], 
       mean_MC[4] + s2*eig_MC_plot.vectors[:, nb_channels-1][4]*eig_MC_plot.values[nb_channels-1]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:dash)

plot!([mean_MC[3] - 10* dgs_dCaN, mean_MC[3] + 10*dgs_dCaN],
      [mean_MC[4] + 10*dgs_dCaL, mean_MC[4] - 10*dgs_dCaL],
      arrow=false, color=:firebrick1, linewidth=4, label="", linestyle=:dashdotdot)

xlabel!(L"\bar{g}_{CaL} (mS/cm^2)")
ylabel!(L"\bar{g}_{CaN} (mS/cm^2)")
xlims!((0, 0.1))
ylims!((0, 0.3001))
display(p2b)
savefig(p2b, "./figures/neuromod_gCaLCaN_MC_DIC.pdf")

# **Computing PCA of low frequency tonic spiking neurons using DICs method (Figure 3)**

In [3]:
g_all_spiking = readdlm("./data/g_all_spiking.dat")
g_all_spiking_DIC = readdlm("./data/g_all_spiking_DIC.dat")
g_all_spiking_leak = readdlm("./data/g_all_spiking_leak.dat")
g_all_spiking_many = readdlm("./data/g_all_spiking_many.dat")

# Definition of the number of cells in the random set
const ncells = 500;

In [33]:
# Number of ion channels in the DA model
nb_channels = 6

# Initializing some variables
g_all_spiking_wout_NMDA = zeros(ncells, nb_channels)

# Removing NMDA channels from the PCA analysis since it is constant
j = 1
for i in [1, 2, 3, 4, 5, 7]
    g_all_spiking_wout_NMDA[:, j] = deepcopy(g_all_spiking[:, i])
    j = j + 1
end

# Means on spiking point cloud
mean_spiking = zeros(1, nb_channels)
mean!(mean_spiking, g_all_spiking_wout_NMDA)

# Defining scaling factors and limits for the plots
s1 = 1.8
s2 = 1.3
Na_max = 60
Kd_max = 20
CaL_max = 0.1
CaN_max = 0.3
ERG_max = 0.25
leak_max = 0.02

# Normalizing data before applying PCA
g_all_spiking_norm = deepcopy(g_all_spiking_wout_NMDA)
for i = 1 : nb_channels
    g_all_spiking_norm[:, i] = g_all_spiking_wout_NMDA[:, i] / std(g_all_spiking_wout_NMDA[:, i])
end

# PCA on spiking point cloud
cov_matrix_spiking = cov(g_all_spiking_norm)
eig_spiking = eigen(cov_matrix_spiking, sortby = x -> abs(x))

# Computing PCA components to map back to the non-normalized dataset
eig_spiking_plot = deepcopy(eig_spiking)
for i = 1 : nb_channels
    eig_spiking_plot.vectors[i, :] = eig_spiking_plot.vectors[i, :] *
                                        std(g_all_spiking_wout_NMDA[:, i])
end

In [35]:
p3a = scatter(g_all_spiking[:, 3], g_all_spiking[:, 4], label="", color=:gray70, grid=false,
              yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm,
              markerstrokewidth=0.)

plot!([mean_spiking[3] - s1*eig_spiking_plot.vectors[:, nb_channels][3]*eig_spiking_plot.values[nb_channels], 
       mean_spiking[3] + s1*eig_spiking_plot.vectors[:, nb_channels][3]*eig_spiking_plot.values[nb_channels]],
       [mean_spiking[4] - s1*eig_spiking_plot.vectors[:, nb_channels][4]*eig_spiking_plot.values[nb_channels], 
       mean_spiking[4] + s1*eig_spiking_plot.vectors[:, nb_channels][4]*eig_spiking_plot.values[nb_channels]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:solid)

plot!([mean_spiking[3] - s2*eig_spiking_plot.vectors[:, nb_channels-1][3]*eig_spiking_plot.values[nb_channels-1], 
       mean_spiking[3] + s2*eig_spiking_plot.vectors[:, nb_channels-1][3]*eig_spiking_plot.values[nb_channels-1]],
       [mean_spiking[4] - s2*eig_spiking_plot.vectors[:, nb_channels-1][4]*eig_spiking_plot.values[nb_channels-1], 
       mean_spiking[4] + s2*eig_spiking_plot.vectors[:, nb_channels-1][4]*eig_spiking_plot.values[nb_channels-1]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:dash)

pearson_cor_spiking = cor(g_all_spiking[:, 3], g_all_spiking[:, 4])
str_pearson_cor_spiking = @sprintf "%.2f" pearson_cor_spiking
annotate!(0.1, 0.07, Plots.text(L"r =  %$str_pearson_cor_spiking", :black, :center, 12))

xlabel!(L"\bar{g}_{CaL} (mS/cm^2)")
ylabel!(L"\bar{g}_{CaN} (mS/cm^2)")
xlims!((0, 0.12))
ylims!((0, 0.12))
display(p3a)
savefig(p3a, "./figures/PCA_DA_neuromod_gCaLCaN_spiking.pdf")

In [38]:
p3b = scatter(g_all_spiking_DIC[:, 3], g_all_spiking_DIC[:, 4], label="", markerstrokewidth=0.,
              markershape=:cross, color=:gray70, grid=false, yguidefontsize=18, 
              xguidefontsize=18, legendfontsize=12, margin=5Plots.mm, legend=false)
scatter!(g_all_spiking_leak[:, 3], g_all_spiking_leak[:, 4], label="", 
         color=:gray70, markershape=:utriangle, markerstrokewidth=0.)

xlabel!(L"\bar{g}_{CaL} (mS/cm^2)")
ylabel!(L"\bar{g}_{CaN} (mS/cm^2)")
xlims!((0, 0.12))
ylims!((0, 0.12))
display(p3b)
savefig(p3b, "./figures/neuromod_gCaLCaN_separation_spiking.pdf")

In [40]:
p3c = scatter(g_all_spiking_many[:, 3], g_all_spiking_many[:, 4], label="", 
              zcolor=g_all_spiking_many[:, 7], grid=false, yguidefontsize=18, xguidefontsize=18, 
              legendfontsize=12, margin=5Plots.mm, c=cgrad(:thermal, rev=false), legend=false, 
              markerstrokewidth=0.)

xlabel!(L"\bar{g}_{CaL} (mS/cm^2)")
ylabel!(L"\bar{g}_{CaN} (mS/cm^2)")
xlims!((0, 0.12))
ylims!((0, 0.12))
display(p3c)
savefig(p3c, "./figures/neuromod_many_gCaLCaN_zgleak_spiking.pdf")

# **Computing PCA of neuromodulated neurons using DICs method (Figure 4)**

In [41]:
g_all_triplets = readdlm("./data/g_all_triplets.dat")
g_all_triplets_DIC = readdlm("./data/g_all_triplets_DIC.dat")
g_all_triplets_leak = readdlm("./data/g_all_triplets_leak.dat")

g_all_bursting = readdlm("./data/g_all_bursting.dat")
g_all_bursting_DIC = readdlm("./data/g_all_bursting_DIC.dat")
g_all_bursting_leak = readdlm("./data/g_all_bursting_leak.dat")

burstiness_triplets = readdlm("./data/burstiness_triplets.dat")
burstiness_triplets_DIC = readdlm("./data/burstiness_triplets_DIC.dat")
burstiness_triplets_leak = readdlm("./data/burstiness_triplets_leak.dat")
burstiness_bursting = readdlm("./data/burstiness_bursting.dat")
burstiness_bursting_DIC = readdlm("./data/burstiness_bursting_DIC.dat")
burstiness_bursting_leak = readdlm("./data/burstiness_bursting_leak.dat");

In [42]:
# Defining colors for the color map
mySalmon = RGB{Float64}(243. / 255., 124. / 255., 130. / 255.)
myYellow = RGB{Float64}(253. / 255., 211. / 255., 44. / 255.)
mycmap = ColorScheme([mySalmon, myYellow]);

In [44]:
# Defining the number of conductances
nb_channels = 6

# Defining a matrix containing all conductances
g_all_bar = zeros(ncells, nb_channels*3)
scaling =  [1, 4, 700, 200, 200, 3000]
g_all_spiking_bar = g_all_spiking[1:end, 1:end .∉ [5]]
g_all_triplets_bar = g_all_triplets[1:end, 1:end .∉ [5]]
g_all_bursting_bar = g_all_bursting[1:end, 1:end .∉ [5]]
for i = 1 : nb_channels
    g_all_bar[:, (i-1)*3 + 1] = g_all_spiking_bar[:, i] .* scaling[i]
    g_all_bar[:, (i-1)*3 + 2] = g_all_triplets_bar[:, i] .* scaling[i]
    g_all_bar[:, (i-1)*3 + 3] = g_all_bursting_bar[:, i] .* scaling[i]
end

p4a = bar([1:4:21], mean(g_all_bar[:, 1:3:16], dims=1)', label="", grid=false, yguidefontsize=18, 
          color=:gray70, ytickfontsize=15, xguidefontsize=15, legendfontsize=12, 
          margin=3Plots.mm, yticks=[0, 50],  xrotation=0, xticks=false, 
          yerror=std(g_all_bar[:, 1:3:16], dims=1)', bar_width=1, ylabel="Conductance (mS/cm²)")

bar!([2:4:22], mean(g_all_bar[:, 2:3:17], dims=1)', label="", grid=false, yguidefontsize=18, 
     color=mySalmon, ytickfontsize=15, xguidefontsize=15, legendfontsize=12, 
     margin=3Plots.mm, yticks=[0, 50],  xrotation=0, xticks=false, 
     yerror=std(g_all_bar[:, 2:3:17], dims=1)', bar_width=1)

bar!([3:4:23], mean(g_all_bar[:, 3:3:18], dims=1)', label="", grid=false, yguidefontsize=18, 
     color=myYellow, ytickfontsize=15, xguidefontsize=15, legendfontsize=12, xtickfontsize=25,
     margin=3Plots.mm, yticks=[0, 50],  xrotation=0, yerror=std(g_all_bar[:, 3:3:18], dims=1)', 
     bar_width=1, xticks=(2:4:22, [L"\bar{g}_{Na}", L"\bar{g}_{Kd}", L"\bar{g}_{CaL}", 
                                   L"\bar{g}_{CaN}", L"\bar{g}_{ERG}", L"g_{leak}"]))

xlims!((0., 24.))
ylims!((0, 50))

CC4 = plot(p4a, layout=(1, 1), legend=false, size=(807,500, 300))
display(CC4)
savefig(CC4, "./figures/neuromod_gCaLCaN_gbar.pdf")

In [45]:
g_all_triplets_wout_NMDA = zeros(ncells, nb_channels)
g_all_bursting_wout_NMDA = zeros(ncells, nb_channels)

# Removing NMDA channels from the PCA analysis since it is constant
j = 1
for i in [1, 2, 3, 4, 5, 7]
    g_all_triplets_wout_NMDA[:, j] = deepcopy(g_all_triplets[:, i])
    g_all_bursting_wout_NMDA[:, j] = deepcopy(g_all_bursting[:, i])
    j = j + 1
end

# Means on triplets point cloud
mean_triplets = zeros(1, nb_channels)
mean!(mean_triplets, g_all_triplets_wout_NMDA)

# Means on bursting point cloud
mean_bursting = zeros(1, nb_channels)
mean!(mean_bursting, g_all_bursting_wout_NMDA)

# Defining scaling factors for the plots
s1 = 2
s2 = 2

# Normalizing data before applying PCA
g_all_triplets_norm = deepcopy(g_all_triplets_wout_NMDA)
for i = 1 : nb_channels
    g_all_triplets_norm[:, i] = g_all_triplets_wout_NMDA[:, i] / std(g_all_triplets_wout_NMDA[:, i])
end

g_all_bursting_norm = deepcopy(g_all_bursting_wout_NMDA)
for i = 1 : nb_channels
    g_all_bursting_norm[:, i] = g_all_bursting_wout_NMDA[:, i] / std(g_all_bursting_wout_NMDA[:, i])
end

# PCA on triplets point cloud
cov_matrix_triplets = cov(g_all_triplets_norm)
eig_triplets = eigen(cov_matrix_triplets, sortby = x -> abs(x))

# PCA on bursting point cloud
cov_matrix_bursting = cov(g_all_bursting_norm)
eig_bursting = eigen(cov_matrix_bursting, sortby = x -> abs(x))

eig_triplets_plot = deepcopy(eig_triplets)
for i = 1 : nb_channels
    eig_triplets_plot.vectors[i, :] = eig_triplets_plot.vectors[i, :] *
                                        std(g_all_triplets_wout_NMDA[:, i])
end

eig_bursting_plot = deepcopy(eig_bursting)
for i = 1 : nb_channels
    eig_bursting_plot.vectors[i, :] = eig_bursting_plot.vectors[i, :] *
                                        std(g_all_bursting_wout_NMDA[:, i])
end

In [47]:
p4b = scatter(g_all_spiking[:, 3], g_all_spiking[:, 4], label="", grid=false, yguidefontsize=18, xguidefontsize=18, 
              margin=5Plots.mm, legend=false, color=:gray70, colorbar_title="burstiness",
              colorbar_titlefontsize=18, markerstrokewidth=0.)
scatter!(g_all_triplets[:, 3], g_all_triplets[:, 4], label="", zcolor=burstiness_triplets, 
         c=cgrad(mycmap, rev=false), markerstrokewidth=0.)
scatter!(g_all_bursting[:, 3], g_all_bursting[:, 4], label="", zcolor=burstiness_bursting,
         c=cgrad(mycmap, rev=false), markerstrokewidth=0.)

plot!([mean_spiking[3] - s1*eig_spiking_plot.vectors[:, nb_channels][3]*eig_spiking_plot.values[nb_channels], 
       mean_spiking[3] + s1*eig_spiking_plot.vectors[:, nb_channels][3]*eig_spiking_plot.values[nb_channels]],
       [mean_spiking[4] - s1*eig_spiking_plot.vectors[:, nb_channels][4]*eig_spiking_plot.values[nb_channels], 
       mean_spiking[4] + s1*eig_spiking_plot.vectors[:, nb_channels][4]*eig_spiking_plot.values[nb_channels]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:solid)

plot!([mean_spiking[3] - s2*eig_spiking_plot.vectors[:, nb_channels-1][3]*eig_spiking_plot.values[nb_channels-1], 
       mean_spiking[3] + s2*eig_spiking_plot.vectors[:, nb_channels-1][3]*eig_spiking_plot.values[nb_channels-1]],
       [mean_spiking[4] - s2*eig_spiking_plot.vectors[:, nb_channels-1][4]*eig_spiking_plot.values[nb_channels-1], 
       mean_spiking[4] + s2*eig_spiking_plot.vectors[:, nb_channels-1][4]*eig_spiking_plot.values[nb_channels-1]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:dash)

plot!([mean_triplets[3] - s1*eig_triplets_plot.vectors[:, nb_channels][3]*eig_triplets_plot.values[nb_channels], 
       mean_triplets[3] + s1*eig_triplets_plot.vectors[:, nb_channels][3]*eig_triplets_plot.values[nb_channels]],
       [mean_triplets[4] - s1*eig_triplets_plot.vectors[:, nb_channels][4]*eig_triplets_plot.values[nb_channels], 
       mean_triplets[4] + s1*eig_triplets_plot.vectors[:, nb_channels][4]*eig_triplets_plot.values[nb_channels]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:solid)

plot!([mean_triplets[3] - s2*eig_triplets_plot.vectors[:, nb_channels-1][3]*eig_triplets_plot.values[nb_channels-1], 
       mean_triplets[3] + s2*eig_triplets_plot.vectors[:, nb_channels-1][3]*eig_triplets_plot.values[nb_channels-1]],
       [mean_triplets[4] - s2*eig_triplets_plot.vectors[:, nb_channels-1][4]*eig_triplets_plot.values[nb_channels-1], 
       mean_triplets[4] + s2*eig_triplets_plot.vectors[:, nb_channels-1][4]*eig_triplets_plot.values[nb_channels-1]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:dash)

plot!([mean_bursting[3] - s1*eig_bursting_plot.vectors[:, nb_channels][3]*eig_bursting_plot.values[nb_channels], 
       mean_bursting[3] + s1*eig_bursting_plot.vectors[:, nb_channels][3]*eig_bursting_plot.values[nb_channels]],
       [mean_bursting[4] - s1*eig_bursting_plot.vectors[:, nb_channels][4]*eig_bursting_plot.values[nb_channels], 
       mean_bursting[4] + s1*eig_bursting_plot.vectors[:, nb_channels][4]*eig_bursting_plot.values[nb_channels]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:solid)

plot!([mean_bursting[3] - s2*eig_bursting_plot.vectors[:, nb_channels-1][3]*eig_bursting_plot.values[nb_channels-1], 
       mean_bursting[3] + s2*eig_bursting_plot.vectors[:, nb_channels-1][3]*eig_bursting_plot.values[nb_channels-1]],
       [mean_bursting[4] - s2*eig_bursting_plot.vectors[:, nb_channels-1][4]*eig_bursting_plot.values[nb_channels-1], 
       mean_bursting[4] + s2*eig_bursting_plot.vectors[:, nb_channels-1][4]*eig_bursting_plot.values[nb_channels-1]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:dash)

pearson_cor_spiking = cor(g_all_spiking[:, 3], g_all_spiking[:, 4])
str_pearson_cor_spiking = @sprintf "%.2f" pearson_cor_spiking
annotate!(0.11, 0.05, Plots.text(L"r =  %$str_pearson_cor_spiking", :black, :center, 12))

pearson_cor_triplets = cor(g_all_triplets[:, 3], g_all_triplets[:, 4])
str_pearson_cor_triplets = @sprintf "%.2f" pearson_cor_triplets
annotate!(0.105, 0.115, Plots.text(L"r =  %$str_pearson_cor_triplets", :black, :center, 12))

pearson_cor_bursting = cor(g_all_bursting[:, 3], g_all_bursting[:, 4])
str_pearson_cor_bursting = @sprintf "%.2f" pearson_cor_bursting
annotate!(0.095, 0.20, Plots.text(L"r =  %$str_pearson_cor_bursting", :black, :center, 12))

xlabel!(L"\bar{g}_{CaL} (mS/cm^2)")
ylabel!(L"\bar{g}_{CaN} (mS/cm^2)")
xlims!((0, 0.12))
ylims!((0, 0.3001))
display(p4b)
savefig(p4b, "./figures/PCA_DA_neuromod_gCaLCaN_burstiness.pdf")

In [49]:
p4c = scatter(g_all_spiking_DIC[:, 3], g_all_spiking_DIC[:, 4], label="", legend=false,
              markershape=:cross, color=:gray70, grid=false, yguidefontsize=18, 
              xguidefontsize=18, legendfontsize=12, margin=5Plots.mm,
              colorbar_title="burstiness", colorbar_titlefontsize=18, markerstrokewidth=0.)
scatter!(g_all_spiking_leak[:, 3], g_all_spiking_leak[:, 4], label="", 
         color=:gray70, markershape=:utriangle, markerstrokewidth=0.)
scatter!(g_all_triplets_DIC[:, 3], g_all_triplets_DIC[:, 4], label="", markershape=:cross,
         zcolor=burstiness_triplets_DIC, c=cgrad(mycmap, rev=false), markerstrokewidth=0.)
scatter!(g_all_triplets_leak[:, 3], g_all_triplets_leak[:, 4], label="", markerstrokewidth=0.,
         markershape=:utriangle, zcolor=burstiness_triplets_leak, c=cgrad(mycmap, rev=false))
scatter!(g_all_bursting_DIC[:, 3], g_all_bursting_DIC[:, 4], label="", markerstrokewidth=0.,
         markershape=:cross, zcolor=burstiness_bursting_DIC, c=cgrad(mycmap, rev=false))
scatter!(g_all_bursting_leak[:, 3], g_all_bursting_leak[:, 4], label="", markerstrokewidth=0.,
         markershape=:utriangle, zcolor=burstiness_bursting_leak, c=cgrad(mycmap, rev=false))

xlabel!(L"\bar{g}_{CaL} (mS/cm^2)")
ylabel!(L"\bar{g}_{CaN} (mS/cm^2)")
xlims!((0, 0.12))
ylims!((0, 0.3001))
display(p4c)
savefig(p4c, "./figures/neuromod_gCaLCaN_separation_burstiness.pdf")

# **Plotting neuromodulation paths using DICs method (Figure 5)**

In [50]:
burstiness = readdlm("./data/burstiness_paths.dat")
g_all_init = readdlm("./data/g_all_init_paths.dat")
gCaLs = readdlm("./data/gCaLs_paths.dat")
gCaNs = readdlm("./data/gCaNs_paths.dat");

In [52]:
# Initializing some variables
n = size(gCaNs)[2]

# Plotting the results
p5a = plot(legend=false)

# Looping over all neurons
for j = 1 : n
    # Plotting first spiking neurons
    i_thresh = minimum(findall(burstiness[:, j] .> 0))
    if i_thresh > 1
        plot!(gCaLs[1:i_thresh, j], gCaNs[1:i_thresh, j], label="", grid=false, linewidth=5, 
              yguidefontsize=18, xguidefontsize=18, margin=5Plots.mm, color=:gray70)
    end
    
    # Then plotting bursting neurons
    plot!(gCaLs[i_thresh:end, j], gCaNs[i_thresh:end, j], label="", 
          line_z=burstiness[i_thresh:end, j], grid=false, linewidth=5, clim=(50, 500),
          yguidefontsize=18, xguidefontsize=18, margin=5Plots.mm, c=cgrad(mycmap, rev=false))
end
xlabel!(L"\bar{g}_{CaL} (mS/cm^2)")
ylabel!(L"\bar{g}_{CaN} (mS/cm^2)")
xlims!((0, 0.1))
ylims!((0, 0.30001))
display(p5a)
savefig(p5a, "./figures/neuromod_gCaLCaN_paths.pdf")

In [54]:
# Fixing random seed
Random.seed!(292)

# Retrieving maximal conductances from what had been computed
j = Int(ceil(rand(1, 1)[1] * n))

# Computing the regression line along the neuromodulation path
line_CaLCaN = fit(gCaLs[:, j], gCaNs[:, j], 1)
a0 = line_CaLCaN[0]
a1 = line_CaLCaN[1]
gCaL_min = minimum(gCaLs[:, j])
gCaL_max = maximum(gCaLs[:, j])

# Extracting the maximal ion channel conductances
(gNa, gKd, _, _, gERG, gNMDA, gleak) = g_all_init[j, :]

# Parameter vector for simulations
p = [0., gNa, gKd, gCaL_min, gCaL_max, gERG, gNMDA, gleak, C, a0, a1]

# Initial conditions
V0 = -90.
x0 = [V0, m_inf(V0), h_inf(V0), n_inf(V0), mCaL_inf(V0), mCaN_inf(V0), 0., 0.]

# Simulation
prob = ODEProblem(DA_ODE_neuromod, x0, tspan, p)
sol = solve(prob, verbose=false)

# Removing transient part
V_t = sol[1, :][320:end-150]
t = sol.t[320:end-150]
p5b = plot(t[1:545], V_t[1:545], linewidth=1.5, grid=false, xticks=[], yticks=[], color=:gray70, 
           yguidefontsize=18, xguidefontsize=18, legend=false, axis=false)
plot!(t[545:end], V_t[545:end], linewidth=1.5, grid=false, xticks=[], yticks=[], 
      line_z=range(0, 500, length(t[545:end])), c=cgrad(mycmap, rev=false), yguidefontsize=18, 
      xguidefontsize=18, legend=false, axis=false)
display(p5b)
savefig(p5b, "./figures/neuromod_gCaLCaN_path_simu1.pdf")

In [57]:
# Fixing random seed
Random.seed!(554)

# Retrieving maximal conductances from what had been computed
j = Int(ceil(rand(1, 1)[1] * n))

# Computing the regression line along the neuromodulation path
line_CaLCaN = fit(gCaLs[:, j], gCaNs[:, j], 1)
a0 = line_CaLCaN[0]
a1 = line_CaLCaN[1]
gCaL_min = minimum(gCaLs[:, j])
gCaL_max = maximum(gCaLs[:, j])

# Extracting the maximal ion channel conductances
(gNa, gKd, _, _, gERG, gNMDA, gleak) = g_all_init[j, :]

# Parameter vector for simulations
p = [0., gNa, gKd, gCaL_min, gCaL_max, gERG, gNMDA, gleak, C, a0, a1]

# Initial conditions
V0 = -90.
x0 = [V0, m_inf(V0), h_inf(V0), n_inf(V0), mCaL_inf(V0), mCaN_inf(V0), 0., 0.]

# Simulation
prob = ODEProblem(DA_ODE_neuromod, x0, tspan, p)
sol = solve(prob, verbose=false)

# Removing transient part
V_t = sol[1, :][320:end-20]
t = sol.t[320:end-20]
p5c = plot(t[1:530], V_t[1:530], linewidth=1.5, grid=false, xticks=[], yticks=[], color=:gray70, 
           yguidefontsize=18, xguidefontsize=18, legend=false, axis=false)
plot!(t[530:end], V_t[530:end], linewidth=1.5, grid=false, xticks=[], yticks=[], 
      line_z=range(0, 500, length(t[530:end])), c=cgrad(mycmap, rev=false), yguidefontsize=18, 
      xguidefontsize=18, legend=false, axis=false)
display(p5c)
savefig(p5c, "./figures/neuromod_gCaLCaN_path_simu2.pdf")

In [59]:
# Definition of simulation time (in ms)
const Tfinal = 10054
const tspan  = (0.0, Tfinal)

# Need of reloading the DA model
include("FYON_2022_DA_models.jl")

# Fixing random seed
Random.seed!(448)

# Retrieving maximal conductances from what had been computed
j = Int(ceil(rand(1, 1)[1] * n))

# Computing the regression line along the neuromodulation path
line_CaLCaN = fit(gCaLs[:, j], gCaNs[:, j], 1)
a0 = line_CaLCaN[0]
a1 = line_CaLCaN[1]
gCaL_min = minimum(gCaLs[:, j])
gCaL_max = maximum(gCaLs[:, j])

# Extracting the maximal ion channel conductances
(gNa, gKd, _, _, gERG, gNMDA, gleak) = g_all_init[j, :]

# Parameter vector for simulations
p = [0., gNa, gKd, gCaL_min, gCaL_max, gERG, gNMDA, gleak, C, a0, a1]

# Initial conditions
V0 = -90.
x0 = [V0, m_inf(V0), h_inf(V0), n_inf(V0), mCaL_inf(V0), mCaN_inf(V0), 0., 0.]

# Simulation
prob = ODEProblem(DA_ODE_neuromod, x0, tspan, p)
sol = solve(prob, verbose=false)

# Removing transient part
V_t = sol[1, :][320:end]
t = sol.t[320:end]
p5d = plot(t[1:352], V_t[1:352], linewidth=1.5, grid=false, xticks=[], yticks=[], color=:gray70, 
           yguidefontsize=18, xguidefontsize=18, legend=false, axis=false)
plot!(t[352:end], V_t[352:end], linewidth=1.5, grid=false, xticks=[], yticks=[], 
      line_z=range(0, 500, length(t[352:end])), c=cgrad(mycmap, rev=false), yguidefontsize=18, 
      xguidefontsize=18, legend=false, axis=false)
display(p5d)
savefig(p5d, "./figures/neuromod_gCaLCaN_path_simu3.pdf")

# **Computing PCA on homogeneous scaled neurons using Monte Carlo (Figure supp 1)**

In [60]:
g_all_MC_leak = readdlm("./data/g_all_MC_leak.dat")
f_leak = readdlm("./data/f_leak.dat")
Iapp = readdlm("./data/Iapp.dat")
rheobase = readdlm("./data/rheobase.dat")

# Definition of the number of cells in the random set
const ncells = 200;

In [61]:
# Initializing some variables
g_all_MC_leak_wout_NMDA = zeros(ncells, nb_channels)

# Removing NMDA channels from the PCA analysis since it is constant
j = 1
for i in [1, 2, 3, 4, 5, 7]
    g_all_MC_leak_wout_NMDA[:, j] = deepcopy(g_all_MC_leak[:, i])
    j = j + 1
end

# Means on spiking point cloud
mean_MC_leak = zeros(1, nb_channels)
mean!(mean_MC_leak, g_all_MC_leak_wout_NMDA)

# Defining scaling factors and limits for the plots
s1 = 2
s2 = 2
Na_max = 60
Kd_max = 20
CaL_max = 0.1
CaN_max = 0.3
ERG_max = 0.25
leak_max = 0.02

# Normalizing data before applying PCA
g_all_MC_leak_norm = deepcopy(g_all_MC_leak_wout_NMDA)
for i = 1 : nb_channels
    g_all_MC_leak_norm[:, i] = g_all_MC_leak_wout_NMDA[:, i] / std(g_all_MC_leak_wout_NMDA[:, i])
end

# PCA on spiking point cloud
cov_matrix_MC_leak = cov(g_all_MC_leak_norm)
eig_MC_leak = eigen(cov_matrix_MC_leak, sortby = x -> abs(x))

# Computing PCA components to map back to the non-normalized dataset
eig_MC_leak_plot = deepcopy(eig_MC_leak)
for i = 1 : nb_channels
    eig_MC_leak_plot.vectors[i, :] = eig_MC_leak_plot.vectors[i, :] *
                                        std(g_all_MC_leak_wout_NMDA[:, i])
end

In [64]:
total_var = sum(eig_MC_leak.values)
eig_val_decreasing = reverse(eig_MC_leak.values)
ps1a = plot(eig_val_decreasing, label="", grid=false, yguidefontsize=15, markershape=:diamond, 
            linestyle=:dashdot, xguidefontsize=15, legendfontsize=12, margin=5Plots.mm, 
            xticks=[1, 2, 3, 4, 5, 6], color=:black, markersize=6, linewidth=2, yticks=[0, 7])
ylims!((-0.2, 7))
xlabel!("Component")
ylabel!("Eigenvalue")
display(ps1a)
savefig(ps1a, "./figures/DA_MC_scree_leak.pdf")

In [66]:
CCs1 = scatter_matrix_zcolor(eig_MC_leak_plot, nb_channels, g_all_MC_leak_wout_NMDA, mean_MC, 
                             Na_max*1.1, Kd_max, CaL_max, CaN_max, ERG_max, leak_max, s1, 0, 
                             rheobase, :none)
display(CCs1)
savefig(CCs1, "./figures/DA_MC_matrix_leak.pdf")

In [68]:
# Plotting the I f curves
ps1b = plot()
for i = 1 : ncells
    plot!(Iapp, f_leak[:, i], label="", grid=false, yguidefontsize=15, xguidefontsize=15, 
          legendfontsize=12, margin=5Plots.mm, color=:black, alpha=0.1, linewidth=2, 
          xticks=[-1, 0], yticks=[0, 2])
end
xlabel!(L"I (mA)")
ylabel!(L"f (Hz)")
xlims!((-1, 0))
ylims!((0, 2))
display(ps1b)
savefig(ps1b, "./figures/simu_DA_MC_hyper_leak.pdf")

# **Plotting low frequency tonic spiking neurons generated using the DICs method (Figure supp 2)**

In [70]:
total_var = sum(eig_spiking.values)
eig_val_decreasing = reverse(eig_spiking.values)

ps2 = plot(eig_val_decreasing, label="", grid=false, yguidefontsize=15, markershape=:diamond, 
           linestyle=:dashdot, xguidefontsize=15, legendfontsize=12, margin=5Plots.mm, 
           xticks=[1, 2, 3, 4, 5, 6], color=:black, markersize=6, linewidth=2, yticks=[0, 5])
ylims!((-0.1, 5))
xlabel!("Component")
ylabel!("Eigenvalue")
display(ps2)
savefig(ps2, "./figures/DA_PCA_spiking_scree.pdf")

In [72]:
CCs2 = scatter_matrix(eig_spiking_plot, nb_channels, g_all_spiking_wout_NMDA, mean_spiking, Na_max, 
                      Kd_max, CaL_max, CaN_max, ERG_max, leak_max, s1, s2, :gray70, :none)
display(CCs2)
savefig(CCs2, "./figures/DA_PCA_spiking_matrix.pdf")

# **Computing PCA on low frequency tonic spiking neurons generated using the DICs method, property separated (Figure supp 3&4)**

In [73]:
# Definition of the number of cells in the random set
const ncells = 500;

In [74]:
# Initializing some variables
g_all_spiking_leak_wout_NMDA = zeros(ncells, nb_channels)

# Removing NMDA channels from the PCA analysis since it is constant
j = 1
for i in [1, 2, 3, 4, 5, 7]
    g_all_spiking_leak_wout_NMDA[:, j] = deepcopy(g_all_spiking_leak[:, i])
    j = j + 1
end

# Means on spiking point cloud
mean_spiking_leak = zeros(1, nb_channels)
mean!(mean_spiking_leak, g_all_spiking_leak_wout_NMDA)

# Defining scaling factors and limits for the plots
s1 = 2
s2 = 2
Na_max = 60
Kd_max = 20
CaL_max = 0.1
CaN_max = 0.3
ERG_max = 0.25
leak_max = 0.02

# Normalizing data before applying PCA
g_all_spiking_leak_norm = deepcopy(g_all_spiking_leak_wout_NMDA)
for i = 1 : nb_channels
    g_all_spiking_leak_norm[:, i] = g_all_spiking_leak_wout_NMDA[:, i] / 
                                    std(g_all_spiking_leak_wout_NMDA[:, i])
end

# PCA on spiking point cloud
cov_matrix_spiking_leak = cov(g_all_spiking_leak_norm)
eig_spiking_leak = eigen(cov_matrix_spiking_leak, sortby = x -> abs(x))

# Computing PCA components to map back to the non-normalized dataset
eig_spiking_leak_plot = deepcopy(eig_spiking_leak)
for i = 1 : nb_channels
    eig_spiking_leak_plot.vectors[i, :] = eig_spiking_leak_plot.vectors[i, :] *
                                        std(g_all_spiking_leak_wout_NMDA[:, i])
end

In [77]:
total_var = sum(eig_spiking_leak.values)
eig_val_decreasing = reverse(eig_spiking_leak.values)
ps3 = plot(eig_val_decreasing, label="", grid=false, yguidefontsize=15, markershape=:diamond, 
           linestyle=:dashdot, xguidefontsize=15, legendfontsize=12, margin=5Plots.mm, 
           xticks=[1, 2, 3, 4, 5, 6], color=:black, markersize=6, linewidth=2, yticks=[0, 7])
ylims!((-0.2, 7))
xlabel!("Component")
ylabel!("Eigenvalue")
display(ps3)
savefig(ps3, "./figures/DA_PCA_spiking_scree_leak.pdf")

In [78]:
CCs3 = scatter_matrix(eig_spiking_leak_plot, nb_channels, g_all_spiking_leak_wout_NMDA, 
                      mean_spiking_leak, Na_max, Kd_max, CaL_max, CaN_max, ERG_max, leak_max*1.1, 
                      s1, 0, :gray70, :utriangle)
display(CCs3)
savefig(CCs3, "./figures/DA_PCA_spiking_leak_matrix.pdf")

In [79]:
# Initializing some variables
g_all_spiking_DIC_wout_NMDA = zeros(ncells, nb_channels)

# Removing NMDA channels from the PCA analysis since it is constant
j = 1
for i in [1, 2, 3, 4, 5, 7]
    g_all_spiking_DIC_wout_NMDA[:, j] = deepcopy(g_all_spiking_DIC[:, i])
    j = j + 1
end

# Means on spiking point cloud
mean_spiking_DIC = zeros(1, nb_channels)
mean!(mean_spiking_DIC, g_all_spiking_DIC_wout_NMDA)

# Defining scaling factors and limits for the plots
s1 = 2
s2 = 2
Na_max = 60
Kd_max = 20
CaL_max = 0.1
CaN_max = 0.3
ERG_max = 0.25
leak_max = 0.02

# Normalizing data before applying PCA
g_all_spiking_DIC_norm = deepcopy(g_all_spiking_DIC_wout_NMDA)
for i = 1 : nb_channels
    g_all_spiking_DIC_norm[:, i] = g_all_spiking_DIC_wout_NMDA[:, i] / 
                                    std(g_all_spiking_DIC_wout_NMDA[:, i])
end

# PCA on spiking point cloud
cov_matrix_spiking_DIC = cov(g_all_spiking_DIC_norm)
eig_spiking_DIC = eigen(cov_matrix_spiking_DIC, sortby = x -> abs(x))

# Computing PCA components to map back to the non-normalized dataset
eig_spiking_DIC_plot = deepcopy(eig_spiking_DIC)
for i = 1 : nb_channels
    eig_spiking_DIC_plot.vectors[i, :] = eig_spiking_DIC_plot.vectors[i, :] *
                                        std(g_all_spiking_DIC_wout_NMDA[:, i])
end

In [81]:
total_var = sum(eig_spiking_DIC.values)
eig_val_decreasing = reverse(eig_spiking_DIC.values)
ps4 = plot(eig_val_decreasing, label="", grid=false, yguidefontsize=15, markershape=:diamond, 
           linestyle=:dashdot, xguidefontsize=15, legendfontsize=12, margin=5Plots.mm, 
           xticks=[1, 2, 3, 4, 5, 6], color=:black, markersize=6, linewidth=2, yticks=[0, 3])
ylims!((-0.1, 3))
xlabel!("Component")
ylabel!("Eigenvalue")
display(ps4)
savefig(ps4, "./figures/DA_PCA_spiking_scree_DIC.pdf")

In [83]:
eig_spiking_DIC_plot.vectors[:, nb_channels-1] = eig_spiking_DIC_plot.vectors[:, nb_channels]
eig_spiking_DIC_plot.values[nb_channels-1]=eig_spiking_DIC_plot.values[nb_channels]
CCs4 = scatter_matrix(eig_spiking_DIC_plot, nb_channels, g_all_spiking_DIC_wout_NMDA, 
                      mean_spiking_DIC, Na_max, Kd_max, CaL_max, CaN_max, ERG_max, leak_max, 
                      0, s2/1.5, :gray70, :cross)
display(CCs4)
savefig(CCs4, "./figures/DA_PCA_spiking_DIC_matrix.pdf")

# **Plotting low frequency tonic spiking neurons generated using the DICs method and highlighting Rin (Figure supp 5)**

In [7]:
# Defining threshold voltage
Vth = -55.5

# Definition of the number of cells in the random set
const ncells = 2000;

In [8]:
# Initializing some variables
Rin_spiking_many = zeros(ncells)

# Looping over all points to compute input resistance at threshold voltage
for i = 1 : ncells
    Rin_spiking_many[i] = (g_all_spiking_many[i, 1]*m_inf(Vth)^3*h_inf(Vth) + 
                           g_all_spiking_many[i, 2]*n_inf(Vth)^3 + 
                           g_all_spiking_many[i, 3]*mCaL_inf(Vth)^2 + 
                           g_all_spiking_many[i, 4]*mCaN_inf(Vth) +
                           g_all_spiking_many[i, 5]*o_inf(Vth) +
                           g_all_spiking_many[i, 6]*NMDA_inf(Vth) +
                           g_all_spiking_many[i, 7])^(-1)
end

In [13]:
ps5 = scatter(g_all_spiking_many[:, 3], g_all_spiking_many[:, 4], label="", 
              zcolor=Rin_spiking_many, grid=false, yguidefontsize=18, xguidefontsize=18, 
              legendfontsize=12, margin=5Plots.mm, c=cgrad(:thermal, rev=false), legend=false, 
              markerstrokewidth=0.)

xlabel!(L"\bar{g}_{CaL} (mS/cm^2)")
ylabel!(L"\bar{g}_{CaN} (mS/cm^2)")
xlims!((0, 0.12))
ylims!((0, 0.12))
display(ps5)
savefig(ps5, "./figures/neuromod_many_gCaLCaN_zRin_spiking.pdf")